# Roof Pitch from LiDAR

This notebook outlines an approach to deriving roof pitch from LiDAR data.

In [24]:
import os
from dotenv import load_dotenv, find_dotenv
from geoalchemy2 import Geometry, WKTElement
import geopandas as gpd
import ipyleaflet
import ipyvolume as ipv
import json
import numpy as np
import numpy.linalg as la
import pandas as pd
import pdal
import pyproj
import pythreejs
import scipy
from shapely.geometry import shape
from shapely.ops import transform
from sqlalchemy import *

In [2]:
def angle_between(v1, v2, degrees=False):
    cosang = np.dot(v1, v2)
    sinang = la.norm(np.cross(v1, v2))
    angle_rad = np.arctan2(sinang, cosang)
    multiplier = 1 if not degrees else 57.2958
    return angle_rad * multiplier

In [3]:
# query postgis to retrieve points
load_dotenv(find_dotenv())
conn_vars = ['PG_USER', 'PG_PASS', 'PG_HOST', 'PG_PORT', 'PG_DB']
user, password, host, port, dbname = [os.getenv(var) for var in conn_vars]
conn_string = f'postgresql+psycopg2://{user}:{password}@{host}:{port}/{dbname}'

In [20]:
engine = create_engine(conn_string, connect_args={'connect_timeout': 10})

In [6]:
m = ipyleaflet.Map(center=(40.675648, -73.990721), zoom=16)
dc = ipyleaflet.DrawControl()
m.add_control(dc)

In [7]:
m

Map(center=[40.675648, -73.990721], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title'…

In [14]:
# get site bounds reprojected as epsg 2263
project = pyproj.Transformer.from_proj(
    pyproj.Proj(init='epsg:4326'),
    pyproj.Proj(init='epsg:2263')
)

map_bounds = dc.last_draw['geometry']
site_bounds_object = transform(project.transform, shape(map_bounds))
site_bounds = site_bounds_object.wkt
marker_size = m.zoom * 0.125

/Users/carsten/miniconda3/envs/gis/lib/python3.9/site-packages/pyproj/crs/crs.py:68: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))
/Users/carsten/miniconda3/envs/gis/lib/python3.9/site-packages/pyproj/crs/crs.py:306: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  projstring = _prepare_from_string(" ".join((projstring, projkwargs)))
/Users/carsten/miniconda3/envs/gis/lib/python3.9/site-packages/pyproj/crs/crs.py:68: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the pr

In [15]:
site_bounds

'POLYGON ((986797.1476360431 185704.5426588609, 986928.0794027386 185627.3191867732, 986868.4536384025 185526.7576581136, 986731.6965641689 185603.6165113914, 986797.1476360431 185704.5426588609))'

In order to get eigenvalues and normals needed for classification and mesh separation, the points need to be imported through PDAL rather than straight from PostGIS to a dataframe:

In [23]:
# get lidar points using PDAL
pipeline_def = {
    "pipeline": [
        {
            "type":"readers.pgpointcloud",
            "connection": f'host={host} dbname={dbname} user={user} password={password} port={port}',
            "table": "pointcloud",
            "column": "pa",
            "spatialreference": "EPSG:2263",
            "where": f"PC_Intersects(pa, ST_GeomFromText('{site_bounds}',2263))"
        },
        {
            "type":"filters.crop",
            "polygon": f"{site_bounds}",
            "distance": 500
        },
        {   "type":"filters.hag_nn"},
        {   "type":"filters.eigenvalues",
            "knn":16},
        {   "type":"filters.normal",
            "knn":16}
    ]
}
pipeline = pdal.Pipeline(json.dumps(pipeline_def))
pipeline.validate()
%time n_points = pipeline.execute()

CPU times: user 516 ms, sys: 82.2 ms, total: 598 ms
Wall time: 4.31 s


In [ ]:
arr = pipeline.arrays[0]
description = arr.dtype.descr
cols = [col for col, __ in description]
df = pd.DataFrame({col: arr[col] for col in cols})

In [ ]:
# "zero" all coordinates for visualization
df['X_0'] = df['X'] - df['X'].min()
df['Y_0'] = df['Y'] - df['Y'].min()
df['Z_0'] = df['Z'] - df['Z'].min()


In [ ]:
# z and y coordinates are swapped here to use the orientation convention of ipyvolume
fig = ipv.figure()

# control = pythreejs.OrbitControls(controlling=fig.camera)
# fig.controls = control
# control.autoRotate = True
fig.render_continuous = True

scatter = ipv.scatter(
    df['X_0'].to_numpy(),
    df['Y_0'].to_numpy(), 
    df['Z_0'].to_numpy(), 
    marker='box', 
    size=marker_size/2,
    color='lightgray')

ipv.squarelim()
# ipv.style.box_off()
# ipv.style.axes_off()
ipv.show()

In [ ]:
# filter out trees & vegetation
df['tree'] = (df['Classification']==1) & (df['HeightAboveGround'] >= 2) & (df['Eigenvalue0'] > .3) &  (df['NumberOfReturns'] - df['ReturnNumber'] >= 1)

tree = ipv.scatter(
    df.loc[df['tree'], 'X_0'].to_numpy(),
    df.loc[df['tree'], 'Y_0'].to_numpy(),
    df.loc[df['tree'], 'Z_0'].to_numpy(),
    marker='box', 
    size=marker_size)

nontree = ipv.scatter(
    df.loc[-df['tree'], 'X_0'].to_numpy(),
    df.loc[-df['tree'], 'Y_0'].to_numpy(),
    df.loc[-df['tree'], 'Z_0'].to_numpy(),
    marker='box', 
    size=marker_size/2)

tree.color='darkgreen'
nontree.color='lightgrey'

# turn off the original one
scatter.visible=False


In [ ]:
# preview roof normals
roof_mask = (df['Classification'] == 1) & (df['HeightAboveGround'] > 10) & (df['Eigenvalue0'] <= .03) & (df['NumberOfReturns'] == df['ReturnNumber'])

roof_normals = ipv.quiver(
    df.loc[roof_mask, 'X_0'].to_numpy(),
    df.loc[roof_mask, 'Y_0'].to_numpy(),
    df.loc[roof_mask, 'Z_0'].to_numpy(),
    df.loc[roof_mask, 'NormalX'].to_numpy(),
    df.loc[roof_mask, 'NormalY'].to_numpy(),
    df.loc[roof_mask, 'NormalZ'].to_numpy(),
    size=marker_size * 3)

tree.visible=False    
#nontree.visible=False
nontree.size=marker_size/3
fig.scatters.append(roof_normals)


Finally, calculate the mean pitch of all roof points and return the value in degrees.

In [ ]:
# measure angles on roof normals to check for flat/pitched
unit_z = np.array([0,0,1])
unit_x = np.array([1,0,0])

df["angle_off_vertical"] = df.apply(lambda item: angle_between(unit_z, np.array([item['NormalX'], item['NormalY'], item['NormalZ']]), degrees=True), axis=1)
print(df.loc[roof_mask, 'angle_off_vertical'].mean())